In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Step 1: Read data and compute mid prices
df = pd.read_csv("/Users/prithvivarman/Desktop/Financial Instruments/Assignment 3/options1.csv")
# Here the given Call/Put are mid prices; we copy them for clarity
df['Call_Mid'] = df['Call']
df['Put_Mid']  = df['Put']

# Step 2: Fit D(T) and B(T) for each maturity via linear regression
spot = 100.0
B_vals = {}
D_vals = {}
for T in df['Time to maturity'].unique():
    subset = df[df['Time to maturity'] == T]
    X = subset[['Strike']].values
    Y = (subset['Call_Mid'] - subset['Put_Mid']).values
    reg = LinearRegression().fit(X, Y)
    intercept = reg.intercept_
    slope = reg.coef_[0]
    # Recover D(T) and B(T) from regression (A = S*D, b = -B)
    B_vals[T] = -slope
    D_vals[T] = intercept / spot

# Step 3: Compute parity RHS and error for each row
df['B(T)'] = df['Time to maturity'].map(B_vals)
df['D(T)'] = df['Time to maturity'].map(D_vals)
df['Parity_RHS_fitted'] = spot * df['D(T)'] - df['Strike'] * df['B(T)']
df['Put_Call_Parity_Error_fitted'] = (df['Call_Mid'] - df['Put_Mid']) - df['Parity_RHS_fitted']

# Step 4: Identify the most mispriced option (max |error|)
df['abs_error'] = df['Put_Call_Parity_Error_fitted'].abs()
most_mispriced = df.loc[df['abs_error'].idxmax()]

print(most_mispriced)

Strike                          110.000000
Time to maturity                  0.750000
Call                              5.564298
Put                              13.602112
Call_Mid                          5.564298
Put_Mid                          13.602112
B(T)                              0.970006
D(T)                              0.984826
Parity_RHS_fitted                -8.218035
Put_Call_Parity_Error_fitted      0.180220
abs_error                         0.180220
Name: 34, dtype: float64


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Step 1: Read data and compute mid prices
df = pd.read_csv("options1.csv")
df['Call_Mid'] = df['Call']
df['Put_Mid'] = df['Put']

# Step 2: Fit D(T) and B(T) for each maturity via linear regression
spot = 100.0
B_vals = {}
D_vals = {}
for T in df['Time to maturity'].unique():
    subset = df[df['Time to maturity'] == T]
    X = subset[['Strike']].values
    Y = (subset['Call_Mid'] - subset['Put_Mid']).values
    reg = LinearRegression().fit(X, Y)
    intercept = reg.intercept_
    slope = reg.coef_[0]
    B_vals[T] = -slope
    D_vals[T] = intercept / spot

# Step 3: Compute parity RHS and error
df['B(T)'] = df['Time to maturity'].map(B_vals)
df['D(T)'] = df['Time to maturity'].map(D_vals)
df['Parity_RHS'] = spot * df['D(T)'] - df['Strike'] * df['B(T)']
df['Put_Call_Parity_Error'] = (df['Call_Mid'] - df['Put_Mid']) - df['Parity_RHS']
df['abs_error'] = df['Put_Call_Parity_Error'].abs()

# Step 4: Identify the most mispriced option
most_mispriced = df.loc[df['abs_error'].idxmax()]
print("Most Mispriced Option:\n", most_mispriced)

# Step 5: Generate arbitrage strategy based on mispricing
K = most_mispriced['Strike']
T = most_mispriced['Time to maturity']
C = most_mispriced['Call_Mid']
P = most_mispriced['Put_Mid']
B_T = most_mispriced['B(T)']
D_T = most_mispriced['D(T)']

# Calculate both sides of parity
lhs = C + K * B_T           # Call + PV(K)
rhs = P + spot * D_T        # Put + S_0

print("\nPut-Call Parity Breakdown:")
print(f"Call + PV(K): {lhs:.6f}")
print(f"Put + Spot:   {rhs:.6f}")
print(f"Difference:   {rhs - lhs:.6f}")

# Step 6: Strategy
print("\nArbitrage Strategy:")

if rhs > lhs:
    print("Put + Stock side is overpriced.")
    print("Strategy (Reverse Conversion):")
    print(f"- Sell 1 Put for ${P:.6f}")
    print(f"- Sell 1 Share at Spot (${spot})")
    print(f"- Buy 1 Call for ${C:.6f}")
    print(f"- Invest ${K * B_T:.6f} in risk-free bond maturing to ${K}")
    profit = (P + spot) - (C + K * B_T)
else:
    print("Call + Bond side is overpriced.")
    print("Strategy (Conversion):")
    print(f"- Buy 1 Put for ${P:.6f}")
    print(f"- Buy 1 Share at Spot (${spot})")
    print(f"- Sell 1 Call for ${C:.6f}")
    print(f"- Borrow ${K * B_T:.6f} at risk-free rate (repay ${K} at maturity)")
    profit = (C + K * B_T) - (P + spot)

print(f"\nLocked-in Arbitrage Profit: ${profit:.6f}")


Most Mispriced Option:
 Strike                   110.000000
Time to maturity           0.750000
Call                       5.564298
Put                       13.602112
Call_Mid                   5.564298
Put_Mid                   13.602112
B(T)                       0.970006
D(T)                       0.984826
Parity_RHS                -8.218035
Put_Call_Parity_Error      0.180220
abs_error                  0.180220
Name: 34, dtype: float64

Put-Call Parity Breakdown:
Call + PV(K): 112.264955
Put + Spot:   112.084735
Difference:   -0.180220

Arbitrage Strategy:
Call + Bond side is overpriced.
Strategy (Conversion):
- Buy 1 Put for $13.602112
- Buy 1 Share at Spot ($100.0)
- Sell 1 Call for $5.564298
- Borrow $106.700657 at risk-free rate (repay $110.0 at maturity)

Locked-in Arbitrage Profit: $-1.337158


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# 1. Load the options data
df = pd.read_csv('options1.csv')  # ensure this file is in your working directory

# 2. Estimate discount factors B(T) via regression: C - P = S*D(T) - K*B(T)
#    We only need B(T) = -slope from regressing (C-P) on K.
B_vals = {}
for T in sorted(df['Time to maturity'].unique()):
    subset = df[df['Time to maturity'] == T]
    X = subset[['Strike']].values
    Y = (subset['Call'] - subset['Put']).values
    reg = LinearRegression().fit(X, Y)
    B_T = -reg.coef_[0]
    B_vals[T] = B_T

# 3. Scan for box‐spread arbitrage
arbs = []
for T in sorted(df['Time to maturity'].unique()):
    subset = df[df['Time to maturity'] == T].sort_values('Strike')
    strikes = subset['Strike'].values
    
    for i in range(len(strikes)):
        for j in range(i+1, len(strikes)):
            K1 = strikes[i]
            K2 = strikes[j]
            c1 = subset.loc[subset['Strike']==K1, 'Call'].iloc[0]
            p1 = subset.loc[subset['Strike']==K1, 'Put' ].iloc[0]
            c2 = subset.loc[subset['Strike']==K2, 'Call'].iloc[0]
            p2 = subset.loc[subset['Strike']==K2, 'Put' ].iloc[0]
            
            # Box cost: (C1 - P1) - (C2 - P2)
            box_cost = (c1 - p1) - (c2 - p2)
            # Bond value: (K2 - K1) * B(T)
            bond_value = (K2 - K1) * B_vals[T]
            
            # If box_cost < bond_value, we have arbitrage
            if box_cost < bond_value - 1e-8:
                profit = bond_value - box_cost
                arbs.append({
                    'T': T,
                    'K1': K1,
                    'K2': K2,
                    'Box Cost': box_cost,
                    'Bond Value': bond_value,
                    'Profit': profit
                })

# 4. Report any arbitrage opportunities
if not arbs:
    print("No box‐spread arbitrage found.")
else:
    arb_df = pd.DataFrame(arbs)
    print("Box‐Spread Arbitrage Opportunities:")
    print(arb_df.to_string(index=False, float_format='%.6f'))


In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Step 1: Read data and compute mid prices
df = pd.read_csv("options1.csv")
df['Call_Mid'] = df['Call']
df['Put_Mid'] = df['Put']

# Step 2: Fit D(T) and B(T) for each maturity via linear regression
spot = 100.0
B_vals = {}
D_vals = {}
for T in df['Time to maturity'].unique():
    subset = df[df['Time to maturity'] == T]
    X = subset[['Strike']].values
    Y = (subset['Call_Mid'] - subset['Put_Mid']).values
    reg = LinearRegression().fit(X, Y)
    intercept = reg.intercept_
    slope = reg.coef_[0]
    B_vals[T] = -slope
    D_vals[T] = intercept / spot

# Step 3: Compute parity RHS and error
df['B(T)'] = df['Time to maturity'].map(B_vals)
df['D(T)'] = df['Time to maturity'].map(D_vals)
df['Parity_RHS'] = spot * df['D(T)'] - df['Strike'] * df['B(T)']
df['Put_Call_Parity_Error'] = (df['Call_Mid'] - df['Put_Mid']) - df['Parity_RHS']
df['abs_error'] = df['Put_Call_Parity_Error'].abs()

# Step 4: Identify the most mispriced option
most_mispriced = df.loc[df['abs_error'].idxmax()]
print("Most Mispriced Option:\n", most_mispriced)

# Step 5: Arbitrage logic
K = most_mispriced['Strike']
T = most_mispriced['Time to maturity']
C = most_mispriced['Call_Mid']
P = most_mispriced['Put_Mid']
B_T = most_mispriced['B(T)']
D_T = most_mispriced['D(T)']

# Compute both sides of parity
lhs = C + K * B_T           # Call + PV(K)
rhs = P + spot * D_T        # Put + Spot

print("\nPut-Call Parity Breakdown:")
print(f"Call + PV(K): {lhs:.6f}")
print(f"Put + Spot:   {rhs:.6f}")
print(f"Difference:   {rhs - lhs:.6f}")

# Step 6: Strategy
print("\nArbitrage Strategy:")

if lhs > rhs:
    # Call + Bond is overpriced → sell it
    print("Call + Bond side is overpriced.")
    print("Strategy (Conversion):")
    print(f"- Buy 1 Put for ${P:.6f}")
    print(f"- Buy 1 Share at Spot (${spot})")
    print(f"- Sell 1 Call for ${C:.6f}")
    print(f"- Borrow ${K * B_T:.6f} at risk-free rate (repay ${K} at maturity)")
    profit = lhs - rhs  # Sell expensive - buy cheap
else:
    # Put + Stock is overpriced → sell it
    print("Put + Stock side is overpriced.")
    print("Strategy (Reverse Conversion):")
    print(f"- Sell 1 Put for ${P:.6f}")
    print(f"- Sell 1 Share at Spot (${spot})")
    print(f"- Buy 1 Call for ${C:.6f}")
    print(f"- Invest ${K * B_T:.6f} in risk-free bond maturing to ${K}")
    profit = rhs - lhs  # Sell expensive - buy cheap

print(f"\nLocked-in Arbitrage Profit: ${profit:.6f}")


Most Mispriced Option:
 Strike                   110.000000
Time to maturity           0.750000
Call                       5.564298
Put                       13.602112
Call_Mid                   5.564298
Put_Mid                   13.602112
B(T)                       0.970006
D(T)                       0.984826
Parity_RHS                -8.218035
Put_Call_Parity_Error      0.180220
abs_error                  0.180220
Name: 34, dtype: float64

Put-Call Parity Breakdown:
Call + PV(K): 112.264955
Put + Spot:   112.084735
Difference:   -0.180220

Arbitrage Strategy:
Call + Bond side is overpriced.
Strategy (Conversion):
- Buy 1 Put for $13.602112
- Buy 1 Share at Spot ($100.0)
- Sell 1 Call for $5.564298
- Borrow $106.700657 at risk-free rate (repay $110.0 at maturity)

Locked-in Arbitrage Profit: $0.180220


In [21]:
df2 = pd.read_csv("/Users/prithvivarman/Desktop/Financial Instruments/Assignment 3/options2.csv")
violations = []
for T, grp in df2.groupby('Time to maturity'):
    grp = grp.sort_values('Strike').reset_index(drop=True)
    for i in range(1, len(grp)):
        if grp.loc[i, 'Call'] > grp.loc[i-1, 'Call']:
            violations.append((T, grp.loc[i-1, 'Strike'], grp.loc[i, 'Strike'],
                                grp.loc[i-1, 'Call'], grp.loc[i, 'Call']))
print("Monotonicity violations (T, K1, K2, C(K1), C(K2)):\n", violations)

Monotonicity violations (T, K1, K2, C(K1), C(K2)):
 [(1.25, 135, 140, 10.5488151176356, 11.3390720741519)]


In [13]:
import pandas as pd

# 1. Read options data
df = pd.read_csv('/Users/prithvivarman/Desktop/Financial Instruments/Assignment 3/options3.csv')

# List to accumulate violation records
violations = []

# 2. Iterate through each unique maturity (TTM)
for ttm in sorted(df['TTM'].unique()):
    # 3. Filter and sort by Strike
    subset = df[df['TTM'] == ttm].sort_values('Strike')
    strikes = subset['Strike'].tolist()
    prices = subset['Call'].tolist()
    
    # 4. Compare all pairs of strikes for this maturity
    n = len(strikes)
    for i in range(n):
        for j in range(i+1, n):
            K1, K2 = strikes[i], strikes[j]
            C_K1, C_K2 = prices[i], prices[j]
            diff = C_K1 - C_K2           # C(K1) - C(K2)
            strike_diff = K2 - K1       # K2 - K1
            
            # Check monotonicity condition: should have C(K1) - C(K2) <= K2 - K1
            if diff > strike_diff:
                # 5. Record violation details
                violations.append({
                    'TTM': ttm,
                    'K1': K1,
                    'K2': K2,
                    'C(K1)': C_K1,
                    'C(K2)': C_K2,
                    'C(K1)-C(K2)': diff,
                    'K2-K1': strike_diff,
                    'Violation': True
                })

# 6. Create DataFrame of violations and display it
violations_df = pd.DataFrame(violations)
print("Q3: === Monotonicity Violations (Strike-wise for fixed TTM) ===")
print(violations_df)


Q3: === Monotonicity Violations (Strike-wise for fixed TTM) ===
   TTM   K1   K2      C(K1)      C(K2)  C(K1)-C(K2)  K2-K1  Violation
0  0.5   95  105  35.010461  24.819664    10.190797     10       True
1  0.5  100  105  30.715062  24.819664     5.895398      5       True


In [23]:
import pandas as pd

# Load options data from CSV
df = pd.read_csv('/Users/prithvivarman/Desktop/Financial Instruments/Assignment 3/options4.csv')

# Group the data by strike price
for strike, group in df.groupby('Strike'):
    # Sort options of this strike by time to maturity
    group_sorted = group.sort_values('Time to maturity')
    times = group_sorted['Time to maturity'].tolist()
    prices = group_sorted['Call'].tolist()
    # Compare each pair of options for this strike
    n = len(times)
    for i in range(n):
        for j in range(i+1, n):
            # If a shorter maturity has a higher price than a longer maturity, flag it
            if times[i] < times[j] and prices[i] > prices[j]:
                print("Mispriced option found (maturity monotonicity violation)!")
                print(f"Strike: {strike}")
                print(f"Option 1: T={times[i]}, Price={prices[i]}")
                print(f"Option 2: T={times[j]}, Price={prices[j]}")



Mispriced option found (maturity monotonicity violation)!
Strike: 150
Option 1: T=0.75, Price=22.360239575553
Option 2: T=1.0, Price=21.7190453765427
